In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import numpy as np
import json
import base64
import io
from matplotlib.pyplot import imread
import codecs
from IPython.display import HTML

In [2]:
movies = pd.read_csv('../input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv')

In [3]:
movies.head()

In [4]:
movies.describe()

In [5]:
movies.iloc[10]

In [6]:
movies.shape

In [7]:
movies.index

In [8]:
movies.columns

In [9]:
movies.iloc[17]

In [10]:
movies.dtypes

In [11]:
movies.iloc[24]

In [12]:
movies.head()

In [13]:
movies.iloc[124]

In [14]:
movies['Genre'] = movies['Genre'].str.split(', ')

In [15]:
movies.iloc[25]

In [16]:
plt.subplots(figsize=(12,10))
list1 = []
for i in movies['Genre']:
    list1.extend(i)
ax = pd.Series(list1).value_counts()[:10].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('hls',10))
for i, v in enumerate(pd.Series(list1).value_counts()[:10].sort_values(ascending=True).values): 
    ax.text(.8, i, v,fontsize=12,color='white',weight='bold')
plt.title('Genres ')
plt.show()

In [17]:
genreList = []
for index, row in movies.iterrows():
    genres = row["Genre"]
    
    for Genre in genres:
        if Genre not in genreList:
            genreList.append(Genre)
genreList[:10]

In [18]:
def binary(genre_list):
    binaryList = []
    
    for Genre in genreList:
        if Genre in genre_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList

In [19]:
movies['genres_bin'] = movies['Genre'].apply(lambda x: binary(x))
movies['genres_bin'].head()

In [20]:
movies['Star1'] = movies['Star1'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
movies['Star1'] = movies['Star1'].str.split(',')

In [21]:
plt.subplots(figsize=(12,10))
list1=[]
for i in movies['Star1']:
    list1.extend(i)
ax=pd.Series(list1).value_counts()[:15].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('muted',40))
for i, v in enumerate(pd.Series(list1).value_counts()[:15].sort_values(ascending=True).values): 
    ax.text(.8, i, v,fontsize=10,color='white',weight='bold')
plt.title('Actors with highest appearance')
plt.show()

In [22]:
castList = []
for index, row in movies.iterrows():
    cast = row["Star1"]
    
    for i in cast:
        if i not in castList:
            castList.append(i)

In [23]:
def binary(cast_list):
    binaryList = []
    
    for Genre in castList:
        if Genre in cast_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList

In [24]:
movies['cast_bin'] = movies['Star1'].apply(lambda x: binary(x))
movies['cast_bin'].head()

In [25]:
def xstr(s):
    if s is None:
        return ''
    return str(s)
movies['Director'] = movies['Director'].apply(xstr)

In [26]:
plt.subplots(figsize=(12,10))
ax = movies[movies['Director']!=''].Director.value_counts()[:10].sort_values(ascending=True).plot.barh(width=0.9,color=sns.color_palette('muted',40))
for i, v in enumerate(movies[movies['Director']!=''].Director.value_counts()[:10].sort_values(ascending=True).values): 
    ax.text(.5, i, v,fontsize=12,color='white',weight='bold')
plt.title('Directors with highest movies')
plt.show()

In [27]:
directorList=[]
for i in movies['Director']:
    if i not in directorList:
        directorList.append(i)

In [28]:
def binary(director_list):
    binaryList = []  
    for direct in directorList:
        if direct in director_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList

In [29]:
movies['director_bin'] = movies['Director'].apply(lambda x: binary(x))
movies.head()

In [30]:
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords


In [31]:
plt.subplots(figsize=(12,12))
stop_words = set(stopwords.words('english'))
stop_words.update(',',';','!','?','.','(',')','$','#','+',':','...',' ','')

words=movies['Overview'].dropna().apply(nltk.word_tokenize)
word=[]
for i in words:
    word.extend(i)
word=pd.Series(word)
word=([i for i in word.str.lower() if i not in stop_words])
wc = WordCloud(background_color="black", max_words=2000, stopwords=STOPWORDS, max_font_size= 60,width=1000,height=1000)
wc.generate(" ".join(word))
plt.imshow(wc)
plt.axis('off')
fig=plt.gcf()
fig.set_size_inches(10,10)
plt.show()

In [32]:
movies['Overview'] = movies['Overview'].str.split(' ')
for i,j in zip(movies['Overview'],movies.index):
    list2 = []
    list2 = i
    movies.loc[j,'Overview'] = str(list2)
movies['Overview'] = movies['Overview'].str.strip('[]').str.replace(' ','').str.replace("'",'')
movies['Overview'] = movies['Overview'].str.split(',')
for i,j in zip(movies['Overview'],movies.index):
    list2 = []
    list2 = i
    list2.sort()
    movies.loc[j,'Overview'] = str(list2)


In [33]:
words_list = []
for index, row in movies.iterrows():
    genres = row["Overview"]
    
    for Genre in genres:
        if Genre not in words_list:
            words_list.append(Genre)

In [34]:
def binary(words):
    binaryList = []
    for Genre in words_list:
        if Genre in words:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList

In [35]:
movies['words_bin'] = movies['Overview'].apply(lambda x: binary(x))
movies = movies[(movies['IMDB_Rating']!=0)] #removing the movies with 0 score and without drector names 
movies = movies[movies['Director']!='']

In [36]:
from scipy import spatial

def Similarity_Between_the_movies_cosine(mov1, mov2):
    movie1 = movies.iloc[mov1]
    movie2 = movies.iloc[mov2]
    
    genresA = movie1['genres_bin']
    genresB = movie2['genres_bin']
    
    genreDistance = spatial.distance.cosine(genresA, genresB)
    
    scoreA = movie1['cast_bin']
    scoreB = movie2['cast_bin']
    scoreDistance = spatial.distance.cosine(scoreA, scoreB)
    
    directA = movie1['director_bin']
    directB = movie2['director_bin']
    directDistance = spatial.distance.cosine(directA, directB)
    
    wordsA = movie1['words_bin']
    wordsB = movie1['words_bin']
    wordsDistance = spatial.distance.cosine(directA, directB)
    return genreDistance + directDistance + scoreDistance + wordsDistance

In [37]:
Similarity_Between_the_movies_cosine(18,24)

In [38]:
print(movies.iloc[18])
print(movies.iloc[24])

In [39]:
new_id = list(range(0,movies.shape[0]))
movies['new_id']=new_id
movies=movies[['Series_Title','Genre','IMDB_Rating','genres_bin','cast_bin','new_id','Director','director_bin','words_bin']]
movies.head()

In [40]:
import operator

def score_prediction_cosine(name):
    #name = input('Enter a movie title: ')
    new_movie = movies[movies['Series_Title'].str.contains(name)].iloc[0].to_frame().T
    print('Selected Movie: ',new_movie.Series_Title.values[0])
    def getNeighbors(baseMovie, K):
        distances = []
    
        for index, movie in movies.iterrows():
            if movie['new_id'] != baseMovie['new_id'].values[0]:
                dist = Similarity_Between_the_movies_cosine(baseMovie['new_id'].values[0], movie['new_id'])
                distances.append((movie['new_id'], dist))
    
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
    
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors

    K = 10
    avgRating = 0
    neighbors = getNeighbors(new_movie, K)
    
    print('\nRecommended Movies: \n')
    for neighbor in neighbors:
        avgRating = avgRating+movies.iloc[neighbor[0]][2]  
        print( movies.iloc[neighbor[0]][0]+" | Genres: "+str(movies.iloc[neighbor[0]][1]).replace(' ','')+" | Rating: "+str(movies.iloc[neighbor[0]][2]))
    
    print('\n')
    avgRating = avgRating/K
    print('The predicted rating for %s is: %f' %(new_movie['Series_Title'].values[0],avgRating))
    print('The actual rating for %s is %f' %(new_movie['Series_Title'].values[0],new_movie['IMDB_Rating']))


In [41]:
score_prediction_cosine('The Thing')

In [42]:
from scipy.spatial.distance import hamming
def Hamming_similarity(mov1, mov2):
    movie1 = movies.iloc[mov1]
    movie2 = movies.iloc[mov2]
    
    genresA = movie1['genres_bin']
    genresB = movie2['genres_bin']
    
    genreDistance = hamming(genresA, genresB)
    
    scoreA = movie1['cast_bin']
    scoreB = movie2['cast_bin']
    scoreDistance = hamming(scoreA, scoreB)
    
    directA = movie1['director_bin']
    directB = movie2['director_bin']
    directDistance = hamming(directA, directB)
    
    wordsA = movie1['words_bin']
    wordsB = movie1['words_bin']
    wordsDistance = hamming(directA, directB)
    return genreDistance + directDistance + scoreDistance + wordsDistance


In [43]:
import operator

def score_prediction_hamming(name):
    #name = input('Enter a movie title: ')
    new_movie = movies[movies['Series_Title'].str.contains(name)].iloc[0].to_frame().T
    print('Selected Movie: ',new_movie.Series_Title.values[0])
    def getNeighbors(baseMovie, K):
        distances = []
    
        for index, movie in movies.iterrows():
            if movie['new_id'] != baseMovie['new_id'].values[0]:
                dist = Hamming_similarity(baseMovie['new_id'].values[0], movie['new_id'])
                distances.append((movie['new_id'], dist))
    
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
    
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors

    K = 10
    avgRating = 0
    neighbors = getNeighbors(new_movie, K)
    
    print('\nRecommended Movies: \n')
    for neighbor in neighbors:
        avgRating = avgRating+movies.iloc[neighbor[0]][2]  
        print( movies.iloc[neighbor[0]][0]+" | Genres: "+str(movies.iloc[neighbor[0]][1]).replace(' ','')+" | Rating: "+str(movies.iloc[neighbor[0]][2]))
    
    print('\n')
    avgRating = avgRating/K
    print('The predicted rating for %s is: %f' %(new_movie['Series_Title'].values[0],avgRating))
    print('The actual rating for %s is %f' %(new_movie['Series_Title'].values[0],new_movie['IMDB_Rating']))


In [44]:
score_prediction_hamming('The Thing')

In [45]:
from scipy.spatial.distance import cityblock
def manhattan_similarity(mov1, mov2):
    movie1 = movies.iloc[mov1]
    movie2 = movies.iloc[mov2]
    
    genresA = movie1['genres_bin']
    genresB = movie2['genres_bin']
    
    genreDistance = cityblock(genresA, genresB)
    
    scoreA = movie1['cast_bin']
    scoreB = movie2['cast_bin']
    scoreDistance = cityblock(scoreA, scoreB)
    
    directA = movie1['director_bin']
    directB = movie2['director_bin']
    directDistance = cityblock(directA, directB)
    
    wordsA = movie1['words_bin']
    wordsB = movie1['words_bin']
    wordsDistance = cityblock(directA, directB)
    return genreDistance + directDistance + scoreDistance + wordsDistance

In [46]:
import operator

def score_prediction_manhattan(name):
    #name = input('Enter a movie title: ')
    new_movie = movies[movies['Series_Title'].str.contains(name)].iloc[0].to_frame().T
    print('Selected Movie: ',new_movie.Series_Title.values[0])
    def getNeighbors(baseMovie, K):
        distances = []
    
        for index, movie in movies.iterrows():
            if movie['new_id'] != baseMovie['new_id'].values[0]:
                dist = manhattan_similarity(baseMovie['new_id'].values[0], movie['new_id'])
                distances.append((movie['new_id'], dist))
    
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
    
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors

    K = 10
    avgRating = 0
    neighbors = getNeighbors(new_movie, K)
    
    print('\nRecommended Movies: \n')
    for neighbor in neighbors:
        avgRating = avgRating+movies.iloc[neighbor[0]][2]  
        print( movies.iloc[neighbor[0]][0]+" | Genres: "+str(movies.iloc[neighbor[0]][1]).replace(' ','')+" | Rating: "+str(movies.iloc[neighbor[0]][2]))
    
    print('\n')
    avgRating = avgRating/K
    print('The predicted rating for %s is: %f' %(new_movie['Series_Title'].values[0],avgRating))
    print('The actual rating for %s is %f' %(new_movie['Series_Title'].values[0],new_movie['IMDB_Rating']))

In [47]:
score_prediction_manhattan('The Thing')

In [48]:
import numpy as np
def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    total=0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        error=((predicted-true)/true)*100
        total=total+error
    # compute the accuracy
    k= abs(total)
    accuracy=100-k
    return accuracy
 

In [49]:
#cosine similarity
y_true = [8.10,8.00,7.9,7.7,8.2,8.00,8.10,8.00,8.10,8.00]
y_pred = [8.9,8.05,7.94,7.98,7.93,7.95,7.92,7.95,8.12,7.97]
# get the accuracy
compute_accuracy(y_true, y_pred)

In [50]:
#hamming 
y_true = [8.10,8.00,7.90,7.70,8.2,8.00,8.10,8.00,8.10,8.00]
y_pred = [8.12,8.24,7.96,7.95,7.93,7.95,7.94,7.95,8.07,7.94]
# get the accuracy
compute_accuracy(y_true, y_pred)

In [51]:
#manhattan 
y_true = [8.10,8.00,7.90,7.70,8.2,8.00,8.10,8.00,8.10,8.00]
y_pred = [8.1, 8.03, 7.94, 7.98, 7.93,7.95,7.92,7.95,8.12,8.07]
# get the accuracy
compute_accuracy(y_true, y_pred )

In [52]:
import numpy as np
import matplotlib.pyplot as plt
data = {'Cosine':92.24, 'Hamming':99.61, 'Manhattan':98.8}
c = list(data.keys())
values = list(data.values())
fig = plt.figure(figsize = (10, 5))
# creating the bar plot
plt.bar(c, values, color ='pink', width = 0.4)
plt.xlabel("Different methods")
plt.ylabel("Accuracy")
plt.show()

In [55]:
import matplotlib.pyplot as plt
plt.xticks([0,1, 2, 3, 4, 5,6,7,8,9,10])  
x1 = [8.9,8.05,7.94,7.98,7.93,7.95,7.92,7.95,8.12,7.97]
plt.plot(x1,  label = "Cosine")
x2 = [8.12,8.24,7.96,7.95,7.93,7.95,7.94,7.95,8.07,7.94]
plt.plot(x2,  label = "Hamming")
actual= [8.10,8.00,7.90,7.70,8.2,8.00,8.10,8.00,8.10,8.00]
plt.plot(actual,  label = "Actual rating")
x3=[8.1, 8.03, 7.94, 7.98, 7.93,7.95,7.92,7.95,8.12,8.07]
plt.plot(x3,  label = "Manhattan")
plt.xlabel('No of Movies')
plt.ylabel('Rating')
plt.title('Comparing the rating among cosine similarity ,manhattan and hamming with the actual rating')
plt.legend()
plt.show()